# Data Conversion and Mapping

File data fields can be converted to strongly typed objects using one of the Convert extension methods of the IRow interface.  Documentation can be found [here](https://github.com/pjoiner/DwC-A_dotnet/wiki/Type-Conversion).

We'll utilize the [DwC-A_dotnet.Mapping](https://github.com/pjoiner/DwC-A_dotnet.Mapping) extension library to map IRow fields to a model class.

In [1]:
#r "nuget:DwC-A_dotnet,0.6.0"
#r "nuget:DwC-A_dotnet.Mapping,0.6.1"
#r "nuget:DwC-A_dotnet.Interactive,0.1.8-Pre"

## Model Class

First create the class that you wish to map values to.  In this instance we are going to create an Occurrence class to display the scientific name and coordinates of the observation.

In [1]:
using DwC_A;

var outputPath = "./data/Papilionidae.zip";
var archive = new ArchiveReader(outputPath);

archive

## Class Definition

The next cell loads a class definition to map the data into.  This code can be created by hand or generated using the dotnet tool [dwca-codegen](https://www.nuget.org/packages/dwca-codegen/) which is available on NuGet.  For more information on using [dwca-codegen](https://github.com/pjoiner/DwC-A_dotnet.Mapping/tree/master/src/dwca-codegen) see the [README.md](https://github.com/pjoiner/DwC-A_dotnet.Mapping/blob/master/src/dwca-codegen/README.md) file for that package / repo.

**Note:** Make sure you set the namespace to "" when generating class files from an archive for use in dotnet interactive since namespaces aren't allowed.

In [1]:
#load "./code/Occurrence.cs"

## Mapper

Next we create a mapper and define a method for mapping the fields of the IRow to the class.

In [1]:
using DwC_A.Mapping;
using DwC_A.Terms;
using DwC_A.Extensions;

var mapper = MapperFactory.CreateMapper<Occurrence>((o, row) => {
    o.GbifID = row["http://rs.gbif.org/terms/1.0/gbifID"];
    o.Identifier = row["http://purl.org/dc/terms/identifier"];
    o.ScientificName = row[Terms.scientificName];
    o.DecimalLongitude = row.Convert<double>(Terms.decimalLongitude);
    o.DecimalLatitude = row.Convert<double>(Terms.decimalLatitude);
});

## Map

Finally, we map the data using the Map methods of either the IRow or IFileReader interfaces.

In [1]:
archive.CoreFile
    .Map<Occurrence>(mapper)
    .Take(100)
    .Select(n => new{n.GbifID, n.Identifier, n.ScientificName, n.DecimalLatitude, n.DecimalLongitude})